In [ ]:
import pandas as pd
# 가라! 판다스! 
from datetime import datetime
# 오늘 날짜 가져오는 모듈
enzyme_table = pd.read_csv('/home/koreanraichu/restriction.csv') 
# 일단 자체적으로 구축했음... 저거 종류 개많습니다 ㅠㅠ 

In [ ]:
year = datetime.today().year
month = datetime.today().month
day = datetime.today().day

In [ ]:
enzyme = input('시퀀스를 찾을 제한효소를 입력해주세요: ').strip()
sequence_name = input('Sequence 이름을 입력해주세요: ').strip()
sequence = input('제한효소 site를 찾을 시퀀스를 입력해주세요: ').upper().strip()
# 제한효소 이름 토씨 하나 안 틀리고 입력해야 합니다. 

In [ ]:
# 사실 함수 정의도 try했었는데 형변환이 안되네요. 
if enzyme_table['Enzyme'].isin([enzyme]).any() == True:
    res_find = enzyme_table.sequence[(enzyme_table['Enzyme'] == enzyme)]
    res_find = res_find.to_string(index=False)
    res_find = str(res_find)
    # 효소 이름이 데이터베이스에 있을 경우 검색할 시퀀스 데이터를 가져온다
    res_site = enzyme_table.restriction_site[(enzyme_table['Enzyme'] == enzyme)]
    res_site = res_site.to_string(index=False)
    res_site = str(res_site)
    # 효소 이름이 데이터베이스에 있을 경우 검색하고 대체할 시퀀스 데이터를 가져온다
    cut_feature = enzyme_table.cut_feature[(enzyme_table['Enzyme'] == enzyme)]
    cut_feature = cut_feature.to_string(index=False)
    cut_feature = str(cut_feature)
    # blunt or sticky(나중에 저장 기능 추가할 때 넣을 예정입니다)
else: 
    print("No data in Database")

In [ ]:
def count_func (a,b):
    while a in b:
        global site_count
        loc = b.find(a)
        site_count += 1
        b = b[loc+len(a):]
    return site_count
# Cutter test하다가 여기에도 추가했음... 
def cut_func (a,b):
    while a in b:
        global res_loc # find로 나오는 값
        global res_loc_list
        seq_length = len(sequence)
        loc = b.find(a)
        b = b[loc+len(a):]
        res_loc = len(sequence) - (len(b) + len(a)) + 1
        res_loc_list.append(str(res_loc)) # find로 나오는 위치 목록(slicing에 따른 보정 필요)
    return res_loc_list
# 여기가 위치 관련 함수입니다. (cutter꺼 그대로 가져왔음)

In [ ]:
if enzyme_table['Enzyme'].isin([enzyme]).any() == True:
    pass
else: 
    pass
# 여기는 검색결과가 존재하지 않으면 -1로 나옵니다. (윗 블럭이랑 여기는 넘어가도 되는 부분)

In [ ]:
with open ('Result_{0}-{1}-{2}_{3}-{4}.txt'.format(year,month,day,enzyme,sequence_name),'w',encoding='utf-8') as f: 
    if sequence.find(res_find) != -1:
        site_count = 0
        res_loc = 0
        res_loc_list = []
        cut_count = count_func(res_find,sequence)
        cut_location = cut_func(res_find,sequence)
        sequence = sequence.replace(res_find,res_site)
        res_loc_list = ', '.join(res_loc_list)
        print(enzyme,",",cut_feature)
        print(sequence,cut_count,res_loc_list)
        f.write("{0} | {1} | {2} | {3} times cut\n".format(enzyme,res_site,cut_feature,cut_count))
        f.write("Cut location(bp): {0} \n".format(res_loc_list))
        f.write('Sequence name: {0} \n{1}'.format(sequence_name,sequence))
        f.close()
        # DB에 효소가 있고 일치하는 시퀀스가 있을 때
    elif enzyme_table['Enzyme'].isin([enzyme]).any() == True and sequence.find(res_find) == -1:  
        print("No restriction site in this sequence. ")
        f.write("{0} | {1} | {2} | 0 times cut\n".format(enzyme,res_site,cut_feature))
        f.write('Sequence name: {0} \n'.format(sequence_name))
        f.write("This restricion enzyme never cut this sequence. ")
        f.close()
        # DB에 효소가 있으나 일치하는 시퀀스가 없을 때
    else:
        print("No data in database. ")
        f.write("{0} \n".format(enzyme))
        f.write("This restriction enzyme not entried in database. ")
        f.close()
        # DB에 효소가 없을 때